# このページについて
論文：[単一ループのRMP](https://arxiv.org/abs/2406.00274)をまとめた内容になります．

# 導入
ロバストMDPにおける強化学習は最悪ケースの遷移確率において，最良の方策を獲得するための学習手法です．方策勾配法を用いてロバストMDPを解く場合，二重ループの構造がよく扱われていました．ループは，ある方策に対してコストを一番高くする遷移確率の計算(最大化)を行う内側ループ，方策を評価して更新する外側ループになっていました．一方で内側ループの計算負荷(内側最大化)がかなりあり，$\mathcal{O}(1/\varepsilon^4)$よりも悪い計算量を要求する場合があります．そこでこの研究ではs-rectangular(不確実性集合の性質)の仮定の下で，単一ループのアルゴリズムを開発しました．
**貢献**
* ネストループ(ループ内にループ)がない最初のRMDPにおける方策勾配アルゴリズム
* s-rectangularの仮定をもつRMDPにたいするアルゴリズム内でSOTAな計算量
	* Gradient dominanceの下での$\mathcal{O}(1/\varepsilon^4)$の収束レート

# 準備

## RMDP設定
この研究で導入されている記法とその意味を一覧で示します．

| 記法                                                                                                                              | 意味                                                                                                                     |
| :------------------------------------------------------------------------------------------------------------------------------ | :--------------------------------------------------------------------------------------------------------------------- |
| $\Delta^d$                                                                                                                      | $d$次元の確率単体                                                                                                             |
| $\|\cdot\|$                                                                                                                     | $L_2$ノルム，フロベニウスノルム                                                                                                     |
| $n$                                                                                                                             | $\{1, 2, \ldots, n\}$：自然集合の要素数                                                                                         |
| $(\mathcal{S}, \mathcal{A}, p, c, \gamma, \rho)$                                                                                | MDPタプル                                                                                                                 |
| $\mathcal{S}:=\{1, 2, \cdots, S\}$                                                                                              | **有限**状態空間                                                                                                             |
| $\mathcal{A}:=\{1, 2, \cdots, A\}$                                                                                              | **有限**行動空間                                                                                                             |
| $p = (p_{sa})_{s \in \mathcal{S}, a \in \mathcal{A}} \in (\Delta^S)^{S \times A}$                                               | 状態遷移確率                                                                                                                 |
| $p_{sa} \in \Delta^S$                                                                                                           | $a$を取った後の$s$から$s'$への状態遷移確率ベクトル                                                                                         |
| $c = (c_{sas'})_{s \in \mathcal{S}, a \in \mathcal{A}, s' \in \mathcal{S}}$                                                     | コスト関数                                                                                                                  |
| $\gamma \in (0, 1)$                                                                                                             | 割引率                                                                                                                    |
| $\rho \in \Delta^S$                                                                                                             | 初期状態の分布                                                                                                                |
| $\pi := (\pi_s)_{s \in \mathcal{S}}$                                                                                            | $s$から$a$上の分布への写像(方策)                                                                                                   |
| $\Pi := (\Delta^A)^S$                                                                                                           | 方策空間                                                                                                                   |
| $\mathcal{P} := (\Delta^S)^{S \times A}$                                                                                        | 遷移確率空間                                                                                                                 |
| $\mathcal{N}_{\Pi}(\pi) := \{\tilde{\pi} \mid \langle\tilde{\pi}, \bar{\pi}-\pi\rangle \leq 0, \forall \bar{\pi} \in \Pi\}$     | $\pi$における$\Pi$の法錐                                                                                                      |
| $\mathcal{N}_{\mathcal{P}}(p) := \{\tilde{p} \mid \langle\tilde{p}, \bar{p}-p\rangle \leq 0, \forall \bar{p} \in \mathcal{P}\}$ | $p$における$\mathcal{P}$の法錐                                                                                                |
| $\text{prox}_{\phi, r_1}(\pi) := \text{argmin}_{\tilde{\pi} \in \Pi} \phi(\tilde{\pi}) + r_1\|\tilde{\pi}-\pi\|^2/2$            | $\phi(\cdot)$ の近接作用素                                                                                                   |
| $\text{dist}(v, \mathcal{V}) := \min_{\bar{v} \in \mathcal{V}}\|\bar{v}-v\|$                                                    | ベクトルまたは行列$v$と集合$\mathcal{V}$の間の距離                                                                                      |
| $\phi(\pi)$                                                                                                                     | $\max _{p \in \mathcal{P}} J_{\rho}(\pi, p)$                                                                           |
| $\chi(\pi, p, \bar{\pi}, \bar{p})$                                                                                              | $J_{\rho}(\pi, p)+\frac{r_{1}}{2}\|\pi-\bar{\pi}\|^{2}-\frac{r_{2}}{2}\|p-\bar{p}\|^{2}$                               |
| $\varphi_{\pi}(p, \bar{\pi}, \bar{p})$                                                                                          | $\min _{\pi \in \Pi} \chi(\pi, p, \bar{\pi}, \bar{p})$                                                                 |
| $\varphi_{\pi, p}(\bar{\pi}, \bar{p})$                                                                                          | $\max _{p \in \mathcal{P}} \varphi_{\pi}(p, \bar{\pi}, \bar{p})$                                                       |
| $\varphi_{\pi, p, \bar{\pi}}(\bar{p})$                                                                                          | $\min _{\bar{\pi}} \varphi_{\pi, p}(\bar{\pi}, \bar{p})$                                                               |
| $\varphi_{\pi, p, \bar{p}}(\bar{\pi})$                                                                                          | $\max _{\bar{p}} \varphi_{\pi, p}(\bar{\pi}, \bar{p})$                                                                 |
| $\underline{\chi}$                                                                                                              | $\min _{\bar{\pi}} \max _{\bar{p}} \varphi_{\pi, p}(\bar{\pi}, \bar{p})$                                               |
| $p^{+}(\bar{\pi}, \bar{p})$                                                                                                     | $\text{Proj}_{\mathcal{P}}(p+\sigma \nabla_{p} \chi(\pi(p, \bar{\pi}, \bar{p}), p, \bar{\pi}, \bar{p}))$               |
| $\bar{p}^{+}(\bar{\pi})$                                                                                                        | $\bar{p}+\mu(p(\pi(\bar{\pi}, \bar{p}), \bar{\pi}, \bar{p})-\bar{p})$                                                  |
| $p^{+}(\bar{p})$                                                                                                                | $\text{Proj}_{\mathcal{P}}(p+\sigma \nabla_{p} J_{\rho}(\pi(p, \bar{\pi}; r_1), p))$                                   |
| $\bar{p}(\bar{\pi})$                                                                                                            | $\text{argmax}_{\bar{p}} \min _{\pi \in \Pi} \max _{p \in \mathcal{P}} \chi(\pi, p, \bar{\pi}, \bar{p})$               |
| $\pi(\bar{\pi}, \bar{p})$                                                                                                       | $\text{argmin}_{\pi \in \Pi} \max _{p \in \mathcal{P}} \chi(\pi, p, \bar{\pi}, \bar{p})$                               |
| $d_{\rho}^{\pi, p}: \mathcal{S} \rightarrow [0, 1]$                                                                             | OM：$d_{\rho}^{\pi, p}(s') = (1-\gamma) \sum_{s \in \mathcal{S}} \sum_{t=0}^{\infty} \gamma^t \rho(s) p_{ss'}^{\pi}(t)$ |
| $v^{\pi, p} := [v_1^{\pi, p}, \ldots, v_S^{\pi, p}]^{\top} \in \mathbb{R}^S$                                                    | 状態価値関数配列                                                                                                               |
| $v_s^{\pi, p} = \mathbb{E}_{\pi, p}[\sum_{t=0}^{\infty} \gamma^t c_{s_t a_t s_{t+1}} \mid s_0=s]$                               | 状態価値関数                                                                                                                 |
| $q_{sa}^{\pi, p} = \mathbb{E}_{\pi, p}[\sum_{t=0}^{\infty} \gamma^t c_{s_t a_t s_{t+1}} \mid s_0=s, a_0=a]$                     | 行動価値関数                                                                                                                 |
| $\pi^* = \text{argmin}_{\pi \in \Pi} \mathbb{E}_{\pi, p, s_0 \sim \rho}[\sum_{t=0}^{\infty} \gamma^t c_{s_t a_t s_{t+1}}]$      | 最適方策                                                                                                                   |
RMDPsにおける最適化の目標は，以下のような最適化問題を解く方策を獲得することです．
$$
\begin{equation*}
\min _{\pi \in \Pi} \max _{p \in \mathcal{P}} J_{\rho}(\pi, p):=\sum_{s \in \mathcal{S}} \rho_{s} v_{s}^{\pi, p} \tag{1}
\end{equation*}
$$










# 今後読む論文
不確実性集合をいい感じに作ってそこでSOTAな計算量を達成しようとしている論文が多い
* 楕円，KL-s rectangular不確実性集合において，主双対first oreder updateおよび，近似価値反復の更新を交互におこなうアルゴリズム
	* Grand-Clément & Kroer, 2021; Kumar et al., 2023a; Li & Lan, 2023）。GrandClément & Kroer (2021)


# メモ
* 目的関数が滑らかというのは強い仮定
* 内側ループは，不確実性集合の要素数回×反復回数演算を行わないといけない
# 初見(あるいは中身だけを知らない)ワードたち


モロー吉田平滑化

片側凸性，PL条件，弱みんてぃ変分不等式，正の相互作用支配，KŁ条件


